In [1]:
import onnx
from onnx_tf.backend import prepare
import numpy as np
from PIL import Image




The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



C:\conda\envs\tf115\lib\site-packages\onnx_tf\common\__init__.py:89: UserWarning: onnx_tf.common.get_outputs_names is deprecated. It will be removed in future release. Use TensorflowGraph.get_outputs_names instead.
  warnings.warn(message)


In [2]:
model = onnx.load('pneumonia.onnx')
tf_rep = prepare(model, strict=False)

C:\conda\envs\tf115\lib\site-packages\onnx_tf\common\handler_helper.py:37: UserWarning: Unknown op ConstantFill in domain `ai.onnx`.
  handler.ONNX_OP, handler.DOMAIN or "ai.onnx"))
C:\conda\envs\tf115\lib\site-packages\onnx_tf\common\handler_helper.py:34: UserWarning: Fail to get since_version of ConvInteger in domain `` with max_inclusive_version=9. Set to 1.
  handler.ONNX_OP, handler.DOMAIN, version))
C:\conda\envs\tf115\lib\site-packages\onnx_tf\common\handler_helper.py:34: UserWarning: Fail to get since_version of DequantizeLinear in domain `` with max_inclusive_version=9. Set to 1.
  handler.ONNX_OP, handler.DOMAIN, version))
C:\conda\envs\tf115\lib\site-packages\onnx_tf\common\handler_helper.py:34: UserWarning: Fail to get since_version of GatherND in domain `` with max_inclusive_version=9. Set to 1.
  handler.ONNX_OP, handler.DOMAIN, version))
C:\conda\envs\tf115\lib\site-packages\onnx_tf\common\handler_helper.py:37: UserWarning: Unknown op ImageScaler in domain `ai.onnx`.
  h


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.


In [3]:
img = Image.open('pneumonia.jpeg')
img = img.resize((256, 256))
img = img.convert('RGB')
img_data = np.asarray(img).astype('float32').transpose(2, 0, 1)
img_data.shape, img_data.dtype

((3, 256, 256), dtype('float32'))

In [4]:
result = tf_rep.run(img_data)
result

Outputs(diagnosis=array([[0.05699854, 0.94300145]], dtype=float32))

In [6]:
tf_rep.export_graph("pneumonia.pb")

In [7]:
import tensorflow as tf

In [8]:
graph_def = tf.compat.v1.GraphDef()
labels = ['normal', 'pneumonia']

with tf.io.gfile.GFile('pneumonia.pb', 'rb') as f:
    graph_def.ParseFromString(f.read())
    tf.import_graph_def(graph_def, name='')

In [9]:
with tf.compat.v1.Session() as sess:
    input_tensor_shape = sess.graph.get_tensor_by_name('image:0').shape

print(input_tensor_shape)

(3, 256, 256)


In [10]:
with tf.compat.v1.Session() as sess:
    prob_tensor = sess.graph.get_tensor_by_name('diagnosis:0')
    predictions, = sess.run(prob_tensor, {'image:0': img_data })

In [11]:
predictions

array([0.05699854, 0.94300145], dtype=float32)